# **Import the whole Ventée Globe data in an Excel file**

author: Stefano Romanelli

email: romanelli@lamma.toscana.it

date: 2021-02-20

license for this ipynb: MIT

license for the original data: ?


## Purposes
The following code permits to download xlsx files, containing information about the race, from https://www.vendeeglobe.org/ and merge all the data in a single big Excel file

## Importing modules
It's up to you to install them before!

In [1]:
import pandas as pd
import requests
import datetime

## Path

**Mandatory:** set the path of the directory of your hard drive where to save the Excel files. Unfortunately, for what I can understand, the original files are badly formatted, so it is impossible to create pandas dataframes that directly accesses the file in internet. We need to save them on the hard drive 

In [2]:
url = 'https://www.vendeeglobe.org/download-race-data/'

# CHANGE IT!!! Use / not \. Put / also at the end
dir_path='C:/Users/romanel/Documents/regatta/files/'

## Functions

### is_downlodable
This funcion controls that the requested file is found on the web server

In [3]:
def is_downloadable(url, file_name):
    """
    Does the url contain a downloadable resource
    """
    h = requests.head(url + file_name, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    return True

### xl_read
This function permits to create the "global.xlsx" file.
Basically it saves the file on the HD (as said it is impossible to read the file directly on the server, at least I didn't succed), it creates a DataFrame of each file using columns "B:U" and skipping the firt 4 rows, adds the header (variable "col"), processes and adds 4 columns: 1) date 2) hour 3) latitude in DD 4) longitude in DD, and merge the actual file with the others previously processed

As, during time, files change their formats (e.g. as soon as the first sailor finishes the race, it is added a new "table" in the sheet, or when sailors withdraw, they are still inserted in the dataframe each day), we filter out the rows in which the column Latitude is null (sailor has finished his/her race) or the column "Rank" is null (sailor withdrew). 

It is possible to change the path and the name of the global file.


In [4]:
def xl_read(file_list):    
    
    dfs = []

    for f in file_list:
        date = format(datetime.datetime.strptime(f[12:20], '%Y%m%d'),'%Y-%m-%d')
        hour = format(datetime.datetime.strptime(f[21:27], '%H%M%S'),"%H:%M:%S")
        col=["Rank", "Nation - Sail", "Skipper - crew", "Hour FR", "Latitude", "Longitude", "Heading - 30m.", "Speed - 30m.", "VMG- 30m.", "Distance - 30m.", "Heading - last rep.", "Speed - last rep.", "VMG- last rep.", "Distance - last rep.", "Heading - 24h", "Speed - 24h", "VMG- 24h", "Distance - 24h", "DTF", "DTL"]
        
        r = requests.get(url + f, allow_redirects=True)
        open(dir_path + f, 'wb').write(r.content)

        df = pd.read_excel(
                    dir_path + f,
                    skipfooter=4,
                    usecols=("B:U"),
                    skiprows=4,
                    names=col
                    )

        df["date"]=date
        df["time"]=hour

        df = df[df['Latitude'].notna()]
        df = df[df['Rank'].notna()]

        df["lat"]=df.apply(lambda row: 
             (int(row.Latitude[:row.Latitude.find('°')] ) + 
             int(row.Latitude[row.Latitude.find('°') + 1:row.Latitude.find('.')]) / 60 + 
             int(row.Latitude[row.Latitude.find('.') + 1:row.Latitude.find('\'')]) / 3600) * 
             ns(row), axis=1)

        df["long"]=df.apply(lambda row: 
             (int(row.Longitude[:row.Longitude.find('°')] ) + 
             int(row.Longitude[row.Longitude.find('°') + 1:row.Longitude.find('.')]) / 60 + 
             int(row.Longitude[row.Longitude.find('.') + 1:row.Longitude.find('\'')]) / 3600) * 
             ew(row), axis=1)

        dfs.append(df)            
    
    return pd.concat(dfs).to_excel(dir_path + 'global.xlsx')

### ns & ew
These functions are needed to convert correctly the coordinates from DMS to DD by assigning the minus sign where needed

In [5]:
def ns(row):
    if row['Latitude'][-1:] == 'N':
        return 1
    else:
        return -1

def ew(row):
    if row['Longitude'][-1:] == 'E':
        return 1
    else:
        return -1

## Listing all the files!

### file list 
This is the list where we store the name of all the Excel files to download and process 

In [6]:
file_list = []

### First day
In the first day, the hours in the file name are different from the ones in the other days...

In [7]:
date_frm='20201108'
for i in ('140000', '150000', '170000', '210000'):
    file_name='vendeeglobe_'+date_frm+'_'+i+'.xlsx'
    if is_downloadable(url, file_name):
        file_list.append(file_name)

### Following days

The end_date variable can be changed, but be aware that putting today date can lead to error at the end of the processing vanishing all the process!!!

**Relax, take a coffee or a beer or both ;) smoke a cigarette (don't!) and after some couple of minutes you'll have your global.xlsx file create!**

At the end of this block the xl_read(file_list) call the function to process data

In [8]:
start_date = datetime.date(2020, 11, 9)
delta = datetime.timedelta(days=1)
end_date = datetime.date(2021, 2, 20)

while start_date <= end_date:
    date_frm=start_date.strftime("%Y%m%d")
    for i in ('040000', '080000', '110000', '140000', '170000', '210000'):
        file_name='vendeeglobe_'+date_frm+'_'+i+'.xlsx'
        if is_downloadable(url, file_name):
            file_list.append(file_name)
    start_date += delta
    
xl_read(file_list)